# Edit mesh_hgr file

mesh_hgr variables in NEMO 3.4 are different from NEMO 3.6. In this Notebook I change the ANHA12 mesh_zgr file variable names to match those for NEMO 3.6

In [3]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np

from salishsea_tools import (
    nc_tools,
    viz_tools,
)

%matplotlib inline

In [4]:
folder = '/ocean/brogalla/GEOTRACES/data/bathymetry/cedar/'
file1 = 'ANHA12_mesh_hgr.nc'
file2 = 'ANHA12_mesh_zgr10.nc'
file3 = 'ANHA12_mask.nc'

In [5]:
mesh_hgr = nc.Dataset(folder + file1)
mesh_zgr = nc.Dataset(folder + file2)
mask = nc.Dataset(folder + file3)

In [6]:
mesh_hgr

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    file_name: mesh_hgr.nc
    TimeStamp: 23/06/2016 19:51:56 -0600
    history: Tue Jun 20 14:25:23 2017: ncks -A -v gdept_0,gdepw_0,e3t_0,e3t_ps,e3w_0,mbathy,tmask ANHA12_mesh_zgr.nc ANHA12_mesh_hgr.nc
    NCO: 4.3.7
    dimensions(sizes): x(1632), y(2400), z(50), t(1)
    variables(dimensions): float32 nav_lon(y,x), float32 nav_lat(y,x), float32 nav_lev(z), float64 time_counter(t), float32 glamt(t,y,x), float32 glamu(t,y,x), float32 glamv(t,y,x), float32 glamf(t,y,x), float32 gphit(t,y,x), float32 gphiu(t,y,x), float32 gphiv(t,y,x), float32 gphif(t,y,x), float64 e1t(t,y,x), float64 e1u(t,y,x), float64 e1v(t,y,x), float64 e1f(t,y,x), float64 e2t(t,y,x), float64 e2u(t,y,x), float64 e2v(t,y,x), float64 e2f(t,y,x), float64 ff(t,y,x), float64 e3t_0(t,z), float64 e3t_ps(t,y,x), float64 e3w_0(t,z), float64 gdept_0(t,z), float64 gdepw_0(t,z), int16 mbathy(t,y,x), int8 tmask(t,z,y,x)
    groups: 

In [8]:
mask

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    file_name: mask.nc
    TimeStamp: 23/06/2016 19:51:33 -0600
    dimensions(sizes): x(1632), y(2400), z(50), t(1)
    variables(dimensions): float32 nav_lon(y,x), float32 nav_lat(y,x), float32 nav_lev(z), float64 time_counter(t), int8 tmask(t,z,y,x), int8 umask(t,z,y,x), int8 vmask(t,z,y,x), int8 fmask(t,z,y,x), int8 tmaskutil(t,y,x), int8 umaskutil(t,y,x), int8 vmaskutil(t,y,x), int8 fmaskutil(t,y,x)
    groups: 

### Create new NetCDF file:

In [7]:
# Load structure from sample dataset:
lat = mesh_hgr.variables['nav_lat']
lon = mesh_hgr.variables['nav_lon']
BX = mesh_hgr.dimensions['x']
BY = mesh_hgr.dimensions['y']
BZ = mesh_hgr.dimensions['z']
t = mesh_hgr.dimensions['t']

In [8]:
len(t)

1

In [19]:
# Setup the new NetCDF file:
ncd = nc.Dataset('ANHA12_mesh_hgr10.nc', 'w', zlib=True)
nc_tools.init_dataset_attrs(
    ncd,
    title='ANHA12 mesh_hgr',
    notebook_name='edit_mesh_hgr-NEMO3.6',
    nc_filepath='./ANHA12_mesh_hgr10.nc',
    comment='Edited ANHA12 mesh_hgr file')

ncd.createDimension('x',len(BX))
ncd.createDimension('y',len(BY))
ncd.createDimension('z',len(BZ))
ncd.createDimension('t',None)

file format: NETCDF4
Conventions: CF-1.6
title: ANHA12 mesh_hgr
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/ccar-modeling/analysis-birgit/src/tip/notebooks/ANHA12/edit_mesh_hgr-NEMO3.6.ipynb
references: REQUIRED
history: [2018-03-22 09:49:21] Created netCDF4 zlib=True dataset.
comment: Edited ANHA12 mesh_hgr file


<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 't', size = 0

In [20]:
nc_tools.show_variables(mesh_hgr)

odict_keys(['nav_lon', 'nav_lat', 'nav_lev', 'time_counter', 'glamt', 'glamu', 'glamv', 'glamf', 'gphit', 'gphiu', 'gphiv', 'gphif', 'e1t', 'e1u', 'e1v', 'e1f', 'e2t', 'e2u', 'e2v', 'e2f', 'ff', 'e3t_0', 'e3t_ps', 'e3w_0', 'gdept_0', 'gdepw_0', 'mbathy', 'tmask'])


In [21]:
#--- Mask variables -----
nav_lat = ncd.createVariable('nav_lat', 'float32', ('y','x'))
nav_lat.long_name = 'Latitude'
nav_lat.units = 'degrees_north'
nav_lat[:] = mesh_hgr.variables['nav_lat']

nav_lon = ncd.createVariable('nav_lon', 'float32', ('y','x'))
nav_lon.long_name = 'Longitude'
nav_lon.units = 'degrees_east'
nav_lon[:] = mesh_hgr.variables['nav_lon']

nav_lev = ncd.createVariable('nav_lev', 'float32', ('z'))
nav_lev.long_name = 'Level'
nav_lev.units = 'metres'
nav_lev[:] = mesh_hgr.variables['nav_lev']

time_counter = ncd.createVariable('time_counter', 'float64', ('t'))
time_counter.units = 'seconds'
time_counter.long_name = 'Time axis'
time_counter[:] = mesh_hgr.variables['time_counter']

glamt = ncd.createVariable('glamt', 'float32', ('t','y','x'))
glamt.units = 'glamt'
glamt.long_name = 'glamt'
glamt[:] = mesh_hgr.variables['glamt']

glamu = ncd.createVariable('glamu', 'float32', ('t','y','x'))
glamu.units = 'glamu'
glamu.long_name = 'glamu'
glamu[:] = mesh_hgr.variables['glamu']

glamv = ncd.createVariable('glamv', 'float32', ('t', 'y','x'))
glamv.units = 'glamv'
glamv.long_name = 'glamv'
glamv[:] = mesh_hgr.variables['glamv']

glamf = ncd.createVariable('glamf', 'float32', ('t', 'y','x'))
glamf.units = 'glamf'
glamf.long_name = 'glamf'
glamf[:] = mesh_hgr.variables['glamf']

gphit = ncd.createVariable('gphit', 'float32', ('t','y','x'))
gphit.units = 'gphit'
gphit.long_name = 'gphit'
gphit[:] = mesh_hgr.variables['gphit']

gphiu = ncd.createVariable('gphiu', 'float32', ('t','y','x'))
gphiu.units = 'gphiu'
gphiu.long_name = 'gphiu'
gphiu[:] = mesh_hgr.variables['gphiu']

gphiv = ncd.createVariable('gphiv', 'float32', ('t', 'y','x'))
gphiv.units = 'gphiv'
gphiv.long_name = 'gphiv'
gphiv[:] = mesh_hgr.variables['gphiv']

gphif = ncd.createVariable('gphif', 'float32', ('t', 'y','x'))
gphif.units = 'gphif'
gphif.long_name = 'gphif'
gphif[:] = mesh_hgr.variables['gphif']

e1t = ncd.createVariable('e1t', 'float64', ('t', 'y','x'))
e1t.units = 'e1t'
e1t.long_name = 'e1t'
e1t[:] = mesh_hgr.variables['e1t']

e1u = ncd.createVariable('e1u', 'float64', ('t', 'y','x'))
e1u.units = 'e1u'
e1u.long_name = 'e1u'
e1u[:] = mesh_hgr.variables['e1u']

e1v = ncd.createVariable('e1v', 'float64', ('t', 'y','x'))
e1v.units = 'e1v'
e1v.long_name = 'e1v'
e1v[:] = mesh_hgr.variables['e1v']

e1f = ncd.createVariable('e1f', 'float64', ('t', 'y','x'))
e1f.units = 'e1f'
e1f.long_name = 'e1f'
e1f[:] = mesh_hgr.variables['e1f']

e2t = ncd.createVariable('e2t', 'float64', ('t', 'y','x'))
e2t.units = 'e2t'
e2t.long_name = 'e2t'
e2t[:] = mesh_hgr.variables['e2t']

e2u = ncd.createVariable('e2u', 'float64', ('t', 'y','x'))
e2u.units = 'e2u'
e2u.long_name = 'e2u'
e2u[:] = mesh_hgr.variables['e2u']

e2v = ncd.createVariable('e2v', 'float64', ('t', 'y','x'))
e2v.units = 'e2v'
e2v.long_name = 'e2v'
e2v[:] = mesh_hgr.variables['e2v']

e2f = ncd.createVariable('e2f', 'float64', ('t', 'y','x'))
e2f.units = 'e2f'
e2f.long_name = 'e2f'
e2f[:] = mesh_hgr.variables['e2f']

ff = ncd.createVariable('ff', 'float64', ('t', 'y','x'))
ff.units = 'ff'
ff.long_name = 'ff'
ff[:] = mesh_hgr.variables['ff']

gdept_1d = ncd.createVariable('gdept_1d', 'float64', ('t', 'z'))
gdept_1d.units = 'gdept_1d'
gdept_1d.long_name = 'gdept_1d'
gdept_1d[:] = mesh_hgr.variables['gdept_0']

gdepw_1d = ncd.createVariable('gdepw_1d', 'float64', ('t','z'))
gdepw_1d.units = 'gdepw_1d'
gdepw_1d.long_name = 'gdepw_1d'
gdepw_1d[:] = mesh_hgr.variables['gdepw_0']

e3t_1d = ncd.createVariable('e3t_1d', 'float64', ('t','z'))
e3t_1d.units = 'e3t_1d'
e3t_1d.long_name = 'e3t_1d'
e3t_1d[:] = mesh_hgr.variables['e3t_0']

e3w_1d = ncd.createVariable('e3w_1d', 'float64', ('t','z'))
e3w_1d.units = 'e3w_1d'
e3w_1d.long_name = 'e3w_1d'
e3w_1d[:] = mesh_hgr.variables['e3w_0']

tmask = ncd.createVariable('tmask', 'int8', ('t', 'z','y','x'))
tmask.units = 'tmask'
tmask.long_name = 'tmask'
tmask[:] = mesh_hgr.variables['tmask']

mbathy = ncd.createVariable('mbathy', 'int16', ('t', 'y','x'))
mbathy.units = 'mbathy'
mbathy.long_name = 'mbathy'
mbathy[:] = mesh_hgr.variables['mbathy']

e3t_ps = ncd.createVariable('e3t_ps', 'float64', ('t', 'y','x'))
e3t_ps.units = 'e3t_ps'
e3t_ps.long_name = 'e3t_ps'
e3t_ps[:] = mesh_hgr.variables['e3t_ps']

In [22]:
nc_tools.check_dataset_attrs(ncd)
ncd.close()

Missing value for dataset attribute: references


In [18]:
e3t_1d

<class 'netCDF4._netCDF4.Variable'>
float64 e3t_1d(z)
    units: e3t_1d
    long_name: e3t_1d
unlimited dimensions: 
current shape = (50,)
filling on, default _FillValue of 9.969209968386869e+36 used